## Data Preparation

In [ ]:
import re                                    # Regular Expressions
import pandas as pd                          # DataFrames & Manipulation
from gensim.models.doc2vec import LabeledSentence, Doc2Vec

In [ ]:
train_input = "../data/recipes.tsv.bz2"

# keep empty strings (http://pandas-docs.github.io/pandas-docs-travis/io.html#na-values)
train = pd.read_csv(train_input, delimiter="\t", quoting=3, encoding="utf-8", keep_default_na=False)

print "loaded %d documents." % len(train)

# Doc2Vec Model Training

see http://radimrehurek.com/gensim/models/doc2vec.html

    class gensim.models.doc2vec.Doc2Vec(
        documents=None,          # list of TaggedDocument elements
        dm=1,                    # training algorithm. dm=1: 'distributed memory' (PV-DM).
                                                       otherwise, 'distributed bag of words' (PV-DBOW).
        dbow_words=0,            # 0 (default), if 1, trains word-vectors simultaneous with DBOW doc-vector
        dm_mean=None,            # 0 (default), if 1, use the mean of context word vectors instead of sum.
        dm_concat=0,             # 0 (default), if 1, use concatenation of (all) context vectors (slow).
        dm_tag_count=1,          # 1 (default), expected document tags per document, when using dm_concat mode
        docvecs=None,
        docvecs_mapfile=None,
        comment=None,
        trim_rule=None,
        **kwargs
    )

In [ ]:
def normalize(text):
    norm_text = text.lower()
    
    for char in ['.', '"', ',', '(', ')', '!', '?', ';', ':']:
        norm_text = norm_text.replace(char, ' ' + char + ' ')
    
    return norm_text

In [ ]:
sentences = [LabeledSentence(normalize(text).split(), [i]) for i, text in enumerate(train['instructions'])]

print "%d sentences in corpus" % len(sentences)

In [ ]:
# Import the built-in logging module and configure it so that Word2Vec creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
model = Doc2Vec(sentences, dm=1, dm_mean=1, size=400, min_count=10, window=10, sample=1e-3, iter=5, workers=4)

In [ ]:
model.wv.most_similar("pasta", topn=20)

In [ ]:
recipe_no = 1
ids = model.docvecs.most_similar(recipe_no)
ids

In [ ]:
match = train.loc[[recipe_no]+[id for id, score in ids]][['title','instructions']]
print "\n".join(list(match['instructions']))